## Lab 1: Introduction to Python

### 1.1 Moving from Excel to Python

We're going to jump right into the Python language by replicating what we did with our ACS data in a programming language, rather than in Excel.  We're going to practice with a new table: B07013, which presents geographic mobility by housing tenure.  My hypothesis is that homeowners are less likely to move than renters, and that in Fruitvale, a lower share of homeowners were in their home a year ago than in Alameda county as a whole.

Let's break down what this would entail:

1) Download data from the ACS

2) Identify which Census Tracts I'm using to define Fruitvale

    a) Create a dummy variable!
    
3) Calculate the percent of householders who are living in the same house as one year ago

    a) for owners and renters; and b) in Fruitvale census tracts and Alameda county
    
4) Calculate the associated MOEs (we'll work on that next week, as well as getting our data into presentation format!)

Ready to go?

### 1.2 The Python Difference

One of the challenging things about using open source technologies is that it is rarely presented as a "complete" software package.  If you're working in Excel, you don't need to go outside of the program to insert a "square root of the sum of squares" equation, for example.  It's a function in Excel. With open source software, different functions are created by different programmers, and we often have to "call" in that external program to do what we want. These are often called libraries.

Some important libraries in Python are:

>**numpy**: used for math and logic operations.

>**pandas**: used for the storing and basic handling of data.

>**matplotlib**: used for data visualization, creating plots, graphs, etc.

>**math**: from datascience, a collection of math functions

We install these libraries with the following commands. The abbreviation will be what we use to "call" functions that belong to that library.  As we start to get more sophisticated, we'll call ever more libraries into our Python notebooks.

In [ ]:
import numpy as np
import pandas as pd
import math
from datascience import *

pd.options.display.float_format = '{:.2f}'.format

### 1.3 Introduction to Tables

A table is a fundamental object type for representing data sets. In Python, tables are presented as a collection of ***arrays***, with each array describing a different attribute for every observation in a dataset. 

Concretely, an array is a **collection of values of the same type**, like a column in an Excel spreadsheet. 

<img src="excel_array.jpg" width="300">

In [ ]:
array1=make_array(1, 2, 10, 1000)
array1

When we have an array, we can tell Python to perform the same operation on every value in the array.  For example, in the code below, I'm telling Python to multiple each value in the array by 100.

In [ ]:
array2=array1*100
array2

Tables extend this idea by containing multiple arrays, each one describing a different attribute for every observation in a dataset. In this way, tables allow us to not only store data about many distinct entities (e.g., individual survey data, census tracts, counties) but to also contain several kinds of data about each observation.

### PANDAS

<img src="panda.jpeg" width="300">

No, not those pandas.  In Python, the word "panda" is derived from the term "panel data", an econometrics term for data sets that include observations over multiple time periods for the same individuals. With pandas, we can clean, transform and analyze our data.

The building block of pandas are "series" - a one-dimensional labeled indexed array.  A dataframe is a multi-dimensional table made up of a collection of series.

<img src="Series.png" width="300">

In [ ]:
data = {
    'apples': [3, 2, 0, 1], 
    'oranges': [0, 3, 7, 2]
}

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df=pd.DataFrame(data, index=['June', 'Robert', 'Lily', 'Rebecca'])
df

### 2.0 Working with ACS Data

Python can read in multiple forms of data, but the most common is a .csv file ("comma separated values").  We can easily import the data we downloaded from American Factfinder into Python.  The ".pd" tells Python to call up the panda function (this is like vocabulary - something to learn), to read the file as a csv, the name of the file, and that the delimiter is a comma. (A delimiter is what separates each of the columns, or array values, from one another.)

In [ ]:
pd.read_csv('ACS_17_5YR_B07013_with_ann.csv', delimiter = ',')

So cool!!!!  But, what did it do to the data variable long names? We have two headers, so we need to tell Python to read both as headers, and not the second row as data.  It may also be helpful to give the dataset a name, and then just look at the first few lines, rather than seeing the full dataset.

In [ ]:
ACS_2017_B07013 = pd.read_csv('ACS_17_5YR_B07013_with_ann.csv', delimiter = ',', header=[0,1], index_col=0)

If we want to view a few rows of our dataset, we can type "name of dataframe.head()".  The default is 5, but we can specify as many rows as we want to see.  If we want to see the last few rows, it's "name of dataframe.tail()".

In [ ]:
ACS_2017_B07013.head()

Better!  But, we've now created what's called a "multi-index" dataframe, which is a lot more complicated to work with.  Let's keep things simple for now, and re-read the data in again, this time just choosing one header row.  I'm going to keep the short labels, rather than the long labels, to save myself from having to type the whole variable name.

In [ ]:
ACS_2017_B07013 = pd.read_csv('ACS_17_5YR_B07013_with_ann.csv', delimiter = ',', header=[0], skiprows=[1])

Programmers often use df to signal that the data are in a "dataframe" - we're going to copy this convention so it becomes easier to problem solve.

In [ ]:
df = pd.read_csv('ACS_17_5YR_B07013_with_ann.csv', delimiter = ',', header=[0], skiprows=[1], index_col=0)

In [ ]:
df

Let's learn a bit about how we can look at our data and understand what is in our dataframe.  We can ask for "information" about our dataframe.

In [ ]:
df.info()

What did it do to our FIPS code?  It read it as a number.  Let's convert it back to a string, or "object". Notice in the code below how I refer to the variable name - I first tell Python that I want to make a change to the dataframe df, and then using the [ ] brackets, I say what variable I am using.  Variable names are always in quotation marks.

In [ ]:
df["GEO.id2"]=df["GEO.id2"].apply(str)

It can also be helpful to rename columns - I like coding with the original variable names (I'll show you why in a second), but if it's confusing to you, you can rename them into something that is meaningful to you.

In [ ]:
df.rename(columns={"GEO.id":"fullfips", "GEO.id2":"fips", "GEO.display-label":"label"}, inplace=True)

In [ ]:
print(df.columns)

In [ ]:
type(fips)

In [ ]:
df.shape

In [ ]:
df.describe()

When we call a variable name as part of a dataframe, it becomes just a series.

In [ ]:
fips_column=df["fips"]
fips_column
#type(fips_column)

If we want to create a new dataframe that only includes the fips column, then we need to specify that it's a dataframe, using the [ ] twice.

In [ ]:
fips_column=df[["fips"]]
fips_column
#type(fips_column)

Finally, if we want to select specific rows in a dataframe, we can either select by the row name using ".loc" or by it's numerical index ".iloc".

In [ ]:
county=df.loc["0500000US06001"]
county

In [ ]:
county=df.iloc[1]
county

### 2.1 Creating our Dummy Variable

We're now going to create our dummy variable.  For this, we're going to use **Boolean** logic, and assign a value of **1** where the condition is met, and a **0** where the condition is not met.

Condition statements in programming language are a bit different from assignment statements.  Common condition statements are

**< less than**
   
**<= less than or equivalent to**
   
**> more than**
   
**>= more than or equivalent to**
   
**== equivalent to**

**!= not equivalent to**

If we want to combine more than one conditional statement, we'll use

**& for "and"**

**| for "or"**

The logic for the code below is as follows:
In my dataframe df, create a new variable/column called "fruitvale", make it equal to (note the 1 = rather than the equivalent == sign) where the dataframe value for the fips variable (df["fips"]) is equivalent to a specified fips code or ("|") another specified fips code. If the condition is true assign it a "1", if it is false, assign it a "0".

In [ ]:
df["fruitvale"]=np.where(((df["fips"]=="6001400300") | (df["fips"]=="6001400400")), 1,0)

In [ ]:
df["fruitvale"]

### 2.2 Creating a New Variable

We're cruising!!!  Let's create a new variable, which is the percent of owners who lived in the same house one year ago, compared to the percent of renters who lived in the same house one year ago.

Let's look at our metadata for a minute, and see if we can figure out how we might create these two new variables.

In [ ]:
df['pct_own_samehouse']=df['HD01_VD06'] / df['HD01_VD03']*100
df['pct_rent_samehouse']=df['HD01_VD07'] / df['HD01_VD04']*100

In [ ]:
df

### 2.3 Calculating Values for our Case Study Community

Now, let's calculate the same values just for Fruitvale.

In [ ]:
fruitvale_df = df[df["fruitvale"]==1]

In [ ]:
fruitvale_df

In [ ]:
fruitvale_pop=df.query("fruitvale==1")["HD01_VD01"].sum()

In [ ]:
fruitvale_pop

In [ ]:
fv=df.groupby("fruitvale").sum()

In [ ]:
fv

In [ ]:
fv['pct_own_samehouse']=fv['HD01_VD06'] / fv['HD01_VD03']*100
fv['pct_rent_samehouse']=fv['HD01_VD07'] / fv['HD01_VD04']*100

How about if I wanted to know what share of householders (regardless of tenure) had moved into a census tract from abroad?  Try coding that on your own below.